Mercury with RPQ

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import os
import time

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Parameters
input_size = 224
batch_size = 16
class_subset = 5  # Only use 5 classes
random_matrix = torch.randn(input_size * input_size * 3, 512, device=device)  # RPQ matrix
cache_limit = 5000

# Cache for RPQ signatures
cache = {}
cache_hits = 0
cache_misses = 0
total_computations = 0

# Data transformations
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    "val": transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

# Load TinyImageNet dataset (subset of 80 classes)
data_dir = "./data/tiny-imagenet-200/train"
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms["train"])

subset_classes = list(full_dataset.class_to_idx.keys())[:class_subset]
filtered_samples = [s for s in full_dataset.samples if s[1] < class_subset]
full_dataset.samples = filtered_samples
full_dataset.targets = [s[1] for s in filtered_samples]

# Split dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Model setup
model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, class_subset)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training with RPQ and WS
def train_model_with_rpq_and_ws(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    global cache_hits, cache_misses, total_computations

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("-" * 20)

        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # RPQ: Compute binary signatures
            inputs_flat = inputs.view(inputs.size(0), -1)
            projections = torch.matmul(inputs_flat, random_matrix)
            signatures = (projections > 0).int().tolist()

            # Check cache
            for sig in signatures:
                sig_tuple = tuple(sig)
                if sig_tuple in cache:
                    cache_hits += 1
                else:
                    cache_misses += 1
                    cache[sig_tuple] = True
                    if len(cache) > cache_limit:
                        cache.pop(next(iter(cache)))  # Remove oldest entry

            # Process batch
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Update stats
            total_computations += inputs.size(0)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels).item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        print(f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}")

    print(f"Cache hits: {cache_hits}, Cache misses: {cache_misses}")
    print(f"Total computations: {total_computations}, Cache hit rate: {cache_hits / (cache_hits + cache_misses):.2f}")

# Train the model
train_model_with_rpq_and_ws(model, train_loader, val_loader, criterion, optimizer,200)

Using device: cuda
Epoch 1/200
--------------------
Train Loss: 3.2315, Train Acc: 0.2145
Epoch 2/200
--------------------
Train Loss: 1.7738, Train Acc: 0.2180
Epoch 3/200
--------------------
Train Loss: 1.6224, Train Acc: 0.2905
Epoch 4/200
--------------------
Train Loss: 1.6041, Train Acc: 0.2920
Epoch 5/200
--------------------
Train Loss: 1.4889, Train Acc: 0.3740
Epoch 6/200
--------------------
Train Loss: 1.4215, Train Acc: 0.4040
Epoch 7/200
--------------------
Train Loss: 1.3925, Train Acc: 0.4290
Epoch 8/200
--------------------
Train Loss: 1.3931, Train Acc: 0.4385
Epoch 9/200
--------------------
Train Loss: 1.3701, Train Acc: 0.4370
Epoch 10/200
--------------------
Train Loss: 1.3704, Train Acc: 0.4420
Epoch 11/200
--------------------
Train Loss: 1.3172, Train Acc: 0.4605
Epoch 12/200
--------------------
Train Loss: 1.2895, Train Acc: 0.4650
Epoch 13/200
--------------------
Train Loss: 1.2923, Train Acc: 0.4805
Epoch 14/200
--------------------
Train Loss: 1.2864, 

Train Loss: 0.7180, Train Acc: 0.7190
Epoch 115/200
--------------------
Train Loss: 0.7286, Train Acc: 0.7220
Epoch 116/200
--------------------
Train Loss: 0.6835, Train Acc: 0.7380
Epoch 117/200
--------------------
Train Loss: 0.7159, Train Acc: 0.7270
Epoch 118/200
--------------------
Train Loss: 0.6974, Train Acc: 0.7350
Epoch 119/200
--------------------
Train Loss: 0.6812, Train Acc: 0.7415
Epoch 120/200
--------------------
Train Loss: 0.7023, Train Acc: 0.7385
Epoch 121/200
--------------------
Train Loss: 0.6989, Train Acc: 0.7400
Epoch 122/200
--------------------
Train Loss: 0.6809, Train Acc: 0.7430
Epoch 123/200
--------------------
Train Loss: 0.6773, Train Acc: 0.7350
Epoch 124/200
--------------------
Train Loss: 0.6993, Train Acc: 0.7195
Epoch 125/200
--------------------
Train Loss: 0.7050, Train Acc: 0.7235
Epoch 126/200
--------------------
Train Loss: 0.6719, Train Acc: 0.7410
Epoch 127/200
--------------------
Train Loss: 0.6734, Train Acc: 0.7470
Epoch 128/200

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import os

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Parameters
input_size = 224  # Image size for ResNet
batch_size = 16
class_subset = 5  # Number of classes to use
num_epochs = 200
random_matrix = torch.randn(input_size * input_size * 3, 512, device=device)  # RPQ projection matrix
cache_limit = 5000  # Maximum cache size

# Cache and statistics
cache = {}  # LRU-like cache
cache_hits = 0  # Count of cache hits
cache_misses = 0  # Count of cache misses
skipped_computations = 0  # Count of computations skipped
total_computations = 0  # Total attempted computations


# Cache update function
def update_cache(signature):
    """
    Updates the cache with the given RPQ signature.
    If the signature is found, count it as a cache hit and skip computation.
    Otherwise, add the signature to the cache, removing the oldest if full.
    """
    global cache_hits, cache_misses, skipped_computations
    if signature in cache:
        cache_hits += 1
        skipped_computations += 1  # Skip computation for cache hit
    else:
        cache_misses += 1
        if len(cache) >= cache_limit:
            cache.pop(next(iter(cache)))  # Remove oldest entry (FIFO)
        cache[signature] = True


# Data transformations
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

# Load TinyImageNet dataset
data_dir = "./data/tiny-imagenet-200/train"
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms["train"])

# Filter for the selected number of classes
subset_classes = list(full_dataset.class_to_idx.keys())[:class_subset]
filtered_samples = [s for s in full_dataset.samples if s[1] < class_subset]
full_dataset.samples = filtered_samples
full_dataset.targets = [s[1] for s in filtered_samples]

# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Model setup
model = models.resnet50(pretrained=False)  # ResNet-50 model
model.fc = nn.Linear(model.fc.in_features, class_subset)  # Adjust final layer for the selected classes
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


# Training function
def train_model_with_rpq(model, train_loader, criterion, optimizer, num_epochs):
    """
    Trains the model using RPQ with cache management.
    Produces a report at the end of training with detailed metrics.
    """
    global cache_hits, cache_misses, skipped_computations, total_computations

    total_loss = 0.0  # Accumulate total loss across epochs

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs} in progress...")

        # Training phase
        model.train()
        epoch_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # RPQ Signature Computation
            inputs_flat = inputs.view(inputs.size(0), -1)  # Flatten input images
            projections = torch.matmul(inputs_flat, random_matrix)  # Project inputs
            signatures = (projections > 0).int()  # Convert projections to binary signatures
            
            # Cache check and management
            for sig in signatures:
                sig_tuple = tuple(sig.tolist())  # Convert tensor to tuple for hashing
                update_cache(sig_tuple)

            # Model forward pass and backpropagation
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Update statistics
            total_computations += inputs.size(0)
            epoch_loss += loss.item() * inputs.size(0)

        total_loss += epoch_loss

    # Final report
    print("\n=== Training Report ===")
    print(f"Total epochs: {num_epochs}")
    print(f"Total training loss: {total_loss / len(train_loader.dataset):.4f}")
    print(f"Cache hits: {cache_hits}")
    print(f"Cache misses: {cache_misses}")
    print(f"Skipped computations: {skipped_computations}")
    print(f"Total computations: {total_computations}")
    print(f"Cache hit rate: {cache_hits / (cache_hits + cache_misses):.2f}")
    print(f"Computation skip rate: {skipped_computations / total_computations:.2f}")
    print("========================")


# Train the model
train_model_with_rpq(model, train_loader, criterion, optimizer, num_epochs)


Using device: cuda
Epoch 1/200 in progress...
Epoch 2/200 in progress...
Epoch 3/200 in progress...
Epoch 4/200 in progress...
Epoch 5/200 in progress...
Epoch 6/200 in progress...
Epoch 7/200 in progress...
Epoch 8/200 in progress...
Epoch 9/200 in progress...
Epoch 10/200 in progress...
Epoch 11/200 in progress...
Epoch 12/200 in progress...
Epoch 13/200 in progress...
Epoch 14/200 in progress...
Epoch 15/200 in progress...
Epoch 16/200 in progress...
Epoch 17/200 in progress...
Epoch 18/200 in progress...
Epoch 19/200 in progress...
Epoch 20/200 in progress...
Epoch 21/200 in progress...
Epoch 22/200 in progress...
Epoch 23/200 in progress...
Epoch 24/200 in progress...
Epoch 25/200 in progress...
Epoch 26/200 in progress...
Epoch 27/200 in progress...
Epoch 28/200 in progress...
Epoch 29/200 in progress...
Epoch 30/200 in progress...
Epoch 31/200 in progress...
Epoch 32/200 in progress...
Epoch 33/200 in progress...
Epoch 34/200 in progress...
Epoch 35/200 in progress...
Epoch 36/2